# ============================
# 1) IMPORTS AND GLOBAL UTILS
# ============================

In [6]:
# IMPORTS 

import numpy as np
import pandas as pd
import pywt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# GLOBAL UTILS
def cyclical_encode(series, period):
    """
    Convert cyclical features to sine/cosine components.
    Example: cyclical_encode(months, 12) -> (month_sin, month_cos).
    """
    return (np.sin(2 * np.pi * series / period),
            np.cos(2 * np.pi * series / period))




In [ ]:
def wavelet_denoise(series, wavelet='db4', level=3):
    """
    Denoise a 1D numpy array using wavelet decomposition.
    """
    coeffs = pywt.wavedec(series, wavelet, mode='per', level=level)
    # Estimating noise level from detail coefficients at the chosen level
    sigma = np.median(np.abs(coeffs[-level])) / 0.6745
    uthresh = sigma * np.sqrt(2 * np.log(len(series)))
    # Thresholding
    coeffs[1:] = [pywt.threshold(c, uthresh, mode='soft') for c in coeffs[1:]]
    # Reconstruct
    return pywt.waverec(coeffs, wavelet, mode='per')[:len(series)]


# ============================================
# 2) LOAD AND PREPROCESS DATA (WITH LOG RAIN)
# ============================================

In [8]:
def load_and_preprocess_data(file_path):
    """
    Loads the CSV file, ensures it's sorted by Date, and returns the DataFrame.
    """
    data = pd.read_csv(file_path, parse_dates=['Date'], dayfirst=True)
    data = data.sort_values('Date').reset_index(drop=True)
    return data


In [9]:

def create_features(data):
    """
    Creates cyclical temporal features and wavelet-denoised series.
    For Rainfall, apply log(1 + x) transform first, then wavelet-denoise.
    """
    # ========== 1) Temporal (Cyclical) Features ==========
    data['Month_sin'], data['Month_cos'] = cyclical_encode(data['Date'].dt.month, 12)
    data['Day_sin'], data['Day_cos'] = cyclical_encode(data['Date'].dt.day, 31)
    data['Weekday_sin'], data['Weekday_cos'] = cyclical_encode(data['Date'].dt.dayofweek, 7)

    # ========== 2) Log-Transform Rainfall Before Denoising ==========
    # We'll create a new column "Rainfall_log1p" and denoise that.
    data['Rainfall_log1p'] = np.log1p(data['Rainfall'].values)
    data['Rainfall_denoised'] = wavelet_denoise(data['Rainfall_log1p'].values)

    # ========== 3) Denoise Temperature Columns (no log needed) ==========
    temp_cols = ['MinTemp', 'MaxTemp', '9amTemp', '3pmTemp']
    for col in temp_cols:
        data[f'{col}_denoised'] = wavelet_denoise(data[col].values)

    # We’ll keep a list of target columns.
    # We'll treat "Rainfall" as our special target (the actual column is Rainfall_denoised).
    target_cols = temp_cols + ['Rainfall']

    # ========== 4) Create Lag Features ==========
    lag_window = 7  # You can try bigger, e.g., 14 or 21.
    for col in temp_cols:
        # For temperatures, only use their own denoised lags
        denoised_col = f'{col}_denoised'
        for lag in range(1, lag_window + 1):
            data[f'{denoised_col}_lag{lag}'] = data[denoised_col].shift(lag)

    # For Rainfall, we create lags from the denoised log(1 + x) column:
    # We'll call them Rainfall_Rainfall_lagX for consistency in the pipeline.
    for lag in range(1, lag_window + 1):
        data[f'Rainfall_Rainfall_lag{lag}'] = data['Rainfall_denoised'].shift(lag)

    # Drop rows with NaN introduced by shifting
    data = data.dropna().reset_index(drop=True)

    return data, target_cols



# ============================
# 3) LSTM MODEL DEFINITION
# ============================



In [10]:
def create_lstm_model(input_shape):
    """
    Creates a Bi-LSTM model with some dropout layers.
    Tunable: you can play with the number of units, dropout,
    or consider an Attention mechanism for further improvements.
    """
    model = Sequential([
        Bidirectional(LSTM(128, return_sequences=True, recurrent_dropout=0.2), input_shape=input_shape),
        Dropout(0.3),
        Bidirectional(LSTM(64, recurrent_dropout=0.1)),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model



# ====================================
# 4) TRAIN AND FORECAST FUNCTION
# ====================================

In [12]:
def train_and_forecast(data, target_col, temporal_features, target_cols):
    """
    Trains a Bi-LSTM model for the given target_col (Rainfall or temp columns).
    Includes:
      - Data scaling
      - Sequence creation
      - Train/test split
      - LSTM model training
      - Prediction & performance metrics
      - Plots for train/test predictions
      - 1-year forecast
    """
    # Identify the correct denoised column for the target
    if target_col == 'Rainfall':
        # For rainfall, we used 'Rainfall_denoised' = log(1 + rainfall) after wavelet
        denoised_col = 'Rainfall_denoised'
        # For features, we used 'Rainfall_Rainfall_lagX'
        features = [f'Rainfall_Rainfall_lag{i}' for i in range(1,8)]
    else:
        denoised_col = f'{target_col}_denoised'
        features = [f'{denoised_col}_lag{i}' for i in range(1,8)]

    # Add cyclical temporal features
    features += temporal_features

    # =======================
    # 1) SCALE THE FEATURES
    # =======================
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data[features + [denoised_col]])

    # We'll create sequences of length = 7 (matching the lag window).
    seq_length = 7
    X, y = [], []
    for i in range(seq_length, len(scaled_data)):
        X.append(scaled_data[i-seq_length:i, :len(features)])
        y.append(scaled_data[i, len(features)])  # The last column is the target

    X = np.array(X)
    y = np.array(y)

    # =========================================
    # 2) TRAIN/TEST SPLIT (80% / 20%)
    # =========================================
    split = int(0.8 * len(X))
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    # =========================
    # 3) TRAIN THE LSTM MODEL
    # =========================
    model = create_lstm_model((X_train.shape[1], X_train.shape[2]))
    history = model.fit(
        X_train, y_train,
        epochs=100,
        batch_size=64,
        validation_data=(X_test, y_test),
        callbacks=[EarlyStopping(patience=7, restore_best_weights=True)],
        verbose=0
    )

    # ===================
    # 4) INVERSE SCALING
    # ===================
    def inverse_scale(values, idx):
        """Reverses the MinMaxScaling for the target column index = idx."""
        dummy = np.zeros((len(values), scaled_data.shape[1]))
        dummy[:, idx] = values
        inverted = scaler.inverse_transform(dummy)[:, idx]
        return inverted

    target_idx = scaled_data.shape[1] - 1  # Index of the target in scaled_data

    # Reconstruct the actual target from scaled predictions
    # If it's rainfall, remember we used log(1 + rainfall).
    if target_col == 'Rainfall':
        y_train_denoised = inverse_scale(y_train, target_idx)
        y_test_denoised = inverse_scale(y_test, target_idx)
        train_pred_denoised = inverse_scale(model.predict(X_train).flatten(), target_idx)
        test_pred_denoised = inverse_scale(model.predict(X_test).flatten(), target_idx)

        # Now exponentiate minus one to get real mm
        y_train_actual = np.expm1(y_train_denoised)
        y_test_actual = np.expm1(y_test_denoised)
        train_pred = np.expm1(train_pred_denoised)
        test_pred = np.expm1(test_pred_denoised)
    else:
        y_train_actual = inverse_scale(y_train, target_idx)
        y_test_actual = inverse_scale(y_test, target_idx)
        train_pred = inverse_scale(model.predict(X_train).flatten(), target_idx)
        test_pred = inverse_scale(model.predict(X_test).flatten(), target_idx)

    # ==============================
    # 5) CALCULATE PERFORMANCE METRICS
    # ==============================
    metrics = {
        'Train MAE': mean_absolute_error(y_train_actual, train_pred),
        'Test MAE': mean_absolute_error(y_test_actual, test_pred),
        'Train RMSE': np.sqrt(mean_squared_error(y_train_actual, train_pred)),
        'Test RMSE': np.sqrt(mean_squared_error(y_test_actual, test_pred)),
        'Train R²': r2_score(y_train_actual, train_pred),
        'Test R²': r2_score(y_test_actual, test_pred)
    }

    # ==============================
    # 6) PLOT TRAINING/TEST RESULTS
    # ==============================
    fig = make_subplots(rows=2, cols=1, subplot_titles=(
        f'Training Performance: {target_col}',
        f'Test Performance: {target_col}'
    ))

    # Plot Training
    fig.add_trace(go.Scatter(
        x=data['Date'].iloc[seq_length:split+seq_length],
        y=y_train_actual,
        name='Actual (Train)',
        line=dict(color='#636efa')
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=data['Date'].iloc[seq_length:split+seq_length],
        y=train_pred,
        name='Predicted (Train)',
        line=dict(color='#ef553b', dash='dot')
    ), row=1, col=1)

    # Plot Test
    fig.add_trace(go.Scatter(
        x=data['Date'].iloc[split+seq_length:],
        y=y_test_actual,
        name='Actual (Test)',
        line=dict(color='#636efa')
    ), row=2, col=1)

    fig.add_trace(go.Scatter(
        x=data['Date'].iloc[split+seq_length:],
        y=test_pred,
        name='Predicted (Test)',
        line=dict(color='#ef553b', dash='dot')
    ), row=2, col=1)

    fig.update_layout(
        height=600,
        title_text=f'{target_col} Forecast Performance',
        template='plotly_white',
        margin=dict(l=40, r=40, t=80, b=40),
        legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
    )
    fig.show()

    # ==============================
    # 7) 1-YEAR FORECAST
    # ==============================
    last_sequence = X_test[-1]
    forecast_steps = 365
    forecast_dates = pd.date_range(
        start=data['Date'].iloc[-1] + pd.Timedelta(days=1),
        periods=forecast_steps
    )

    # We'll store the scaled predictions in this list
    forecasts_scaled = []
    current_sequence = last_sequence.copy()

    # We'll need an index map for updating cyclical features
    feat_to_idx = {feat: i for i, feat in enumerate(features)}

    for i in range(forecast_steps):
        # Model prediction (scaled)
        pred_scaled = model.predict(current_sequence[np.newaxis, ...])[0, 0]
        forecasts_scaled.append(pred_scaled)

        # Prepare the next time-step
        new_features = current_sequence[-1].copy()
        current_date = forecast_dates[i]

        # ========== Update cyclical date features ==========
        month_sin, month_cos = cyclical_encode(current_date.month, 12)
        day_sin, day_cos = cyclical_encode(current_date.day, 31)
        weekday_sin, weekday_cos = cyclical_encode(current_date.weekday(), 7)

        new_features[feat_to_idx['Month_sin']] = month_sin
        new_features[feat_to_idx['Month_cos']] = month_cos
        new_features[feat_to_idx['Day_sin']] = day_sin
        new_features[feat_to_idx['Day_cos']] = day_cos
        new_features[feat_to_idx['Weekday_sin']] = weekday_sin
        new_features[feat_to_idx['Weekday_cos']] = weekday_cos

        # ========== Update Lag Features ==========
        if target_col == 'Rainfall':
            # Move each lag back one step
            for lag in range(6, 0, -1):
                current_lag_feat = f'Rainfall_Rainfall_lag{lag}'
                next_lag_feat = f'Rainfall_Rainfall_lag{lag+1}'
                new_features[feat_to_idx[next_lag_feat]] = new_features[feat_to_idx[current_lag_feat]]
            # The new lag1 becomes our freshly predicted scaled log(1 + rainfall)
            new_features[feat_to_idx['Rainfall_Rainfall_lag1']] = pred_scaled
        else:
            # For temperatures:
            denoised_target = f'{target_col}_denoised'
            for lag in range(6, 0, -1):
                current_lag_feat = f'{denoised_target}_lag{lag}'
                next_lag_feat = f'{denoised_target}_lag{lag+1}'
                new_features[feat_to_idx[next_lag_feat]] = new_features[feat_to_idx[current_lag_feat]]
            # The new lag1 becomes our predicted scaled temperature
            new_features[feat_to_idx[f'{denoised_target}_lag1']] = pred_scaled

        # Shift the sequence to incorporate the new row
        current_sequence = np.vstack([current_sequence[1:], new_features])

    # ========== Convert scaled forecast to actual units ==========
    forecasts_scaled = np.array(forecasts_scaled)
    forecasts_denoised = inverse_scale(forecasts_scaled, target_idx)

    if target_col == 'Rainfall':
        # Convert back from log(1 + x)
        forecast_actual = np.expm1(forecasts_denoised)
    else:
        forecast_actual = forecasts_denoised

    # =========================
    # 8) PLOT HISTORICAL + FORECAST
    # =========================
    forecast_fig = go.Figure()
    forecast_fig.add_trace(go.Scatter(
        x=data['Date'],
        y=data[target_col],
        name='Historical',
        line=dict(color='#636efa')
    ))
    forecast_fig.add_trace(go.Scatter(
        x=forecast_dates,
        y=forecast_actual,
        name='Forecast',
        line=dict(color='#00cc96', dash='dot')
    ))
    forecast_fig.update_layout(
        title=f'{target_col} - 1 Year Forecast',
        xaxis_title='Date',
        yaxis_title=target_col,
        template='plotly_white',
        hovermode='x unified',
        legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
    )
    forecast_fig.show()

    
    # ==============================
    # 9) SAVE PREDICTIONS AND FORECAST TO CSV  
    # ==============================
    # Get original rainfall values for actual data
    train_start_idx = seq_length
    train_end_idx = split + seq_length
    test_start_idx = split + seq_length

    train_df = pd.DataFrame({
        'Date': data['Date'].iloc[train_start_idx:train_end_idx],
        # Use original rainfall values for actual data
        'Actual': data['Rainfall'].iloc[train_start_idx:train_end_idx].values,
        'Predicted': train_pred,
        'Type': 'train'
    })

    test_df = pd.DataFrame({
        'Date': data['Date'].iloc[test_start_idx:],
        # Use original rainfall values for actual data
        'Actual': data['Rainfall'].iloc[test_start_idx:].values,
        'Predicted': test_pred,
        'Type': 'test'
    })

    forecast_df = pd.DataFrame({
        'Date': forecast_dates,
        'Actual': np.nan,  # No actual values for forecast period
        'Predicted': forecast_actual,
        'Type': 'forecast'
    })

    # Combine and save
    combined_df = pd.concat([train_df, test_df, forecast_df])
    filename = f"{target_col}_predictions.csv"
    combined_df.to_csv(filename, index=False)
    print(f"\nSaved predictions for {target_col} to {filename}")

    return model, metrics



# =====================
# 5) MAIN EXECUTION
# =====================

In [6]:
if __name__ == "__main__":
    # 1) Load Data
    data = load_and_preprocess_data('data/Cleaned_TemperatureRainFall.csv')
    data, target_cols = create_features(data)

    # 2) Common temporal features
    temporal_features = ['Month_sin', 'Month_cos', 'Day_sin', 'Day_cos', 'Weekday_sin', 'Weekday_cos']

    # 3) Train and Evaluate each target
    results = {}
    for target in target_cols:
        print(f"\n=== Training {target} ===")
        model, metrics = train_and_forecast(data, target, temporal_features, target_cols)
        results[target] = {'model': model, 'metrics': metrics}
        print(pd.Series(metrics))

    # 4) Save metrics
    pd.DataFrame({k: v['metrics'] for k, v in results.items()}).T.to_csv('forecast_metrics.csv')
    print("\nMetrics saved to 'forecast_metrics.csv'.")


=== Training MinTemp ===


c:\Users\home\miniconda3\envs\dash\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



75/75 ━━━━━━━━━━━━━━━━━━━━ 13s 92ms/step
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 274ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/s

Train MAE     0.494237
Test MAE      0.507143
Train RMSE    0.624745
Test RMSE     0.660291
Train R²      0.986792
Test R²       0.983152
dtype: float64

=== Training MaxTemp ===


...

Train MAE     0.707438
Test MAE      0.574639
Train RMSE    0.939289
Test RMSE     0.748583
Train R²      0.981157
Test R²       0.988564
dtype: float64

=== Training Rainfall ===


c:\Users\home\miniconda3\envs\dash\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



75/75 ━━━━━━━━━━━━━━━━━━━━ 13s 103ms/step
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 495ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 365ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

Train MAE     0.151237
Test MAE      0.151345
Train RMSE    0.253768
Test RMSE     0.298835
Train R²      0.923424
Test R²       0.912320
dtype: float64

All Actual, Predicted, and Forecast data saved to 'All_Actual_Predicted_Forecast.csv'.
Metrics saved to 'forecast_metrics.csv'.
